In [3]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pickle
import requests
import json
from sklearn.compose import ColumnTransformer 
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline

In [4]:
# Loading data
file_path = Path("train_test.csv")
df = pd.read_csv(file_path)
df

/Users/kalaimano/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,age,body_type,diet,drinks,drugs,education,ethnicity,height,income,job,orientation,religion,sex,smokes,status,class
0,22,curvy,anything,rarely,drugs_no,noansweredu,asian,75.0,29592,transportation,straight,religious,m,rarely,single,3.0
1,35,average,anything,rarely,drugs_yes,college/university,white,70.0,48630,hospitality / travel,straight,religious,m,no,single,3.0
2,38,skinny,anything,rarely,noanswerdrugs,masters program,noanswerethnicity,68.0,60812,noanswerjob,straight,religious,m,no,available,5.0
3,23,skinny,vegetarian,rarely,noanswerdrugs,noansweredu,white,71.0,18578,student,straight,religious,m,no,single,3.0
4,29,athletic,noanswerdiet,rarely,drugs_no,college/university,asian,66.0,94691,artistic / musical / writer,straight,religious,m,no,single,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59940,59,noanswerbodytype,noanswerdiet,rarely,drugs_no,college/university,noanswerethnicity,62.0,16612,sales / marketing / biz dev,straight,religious,f,no,single,0.0
59941,24,fit,anything,rarely,drugs_yes,noansweredu,white,72.0,118254,entertainment / media,straight,agnostic,m,no,single,NaN
59942,42,average,anything,drinks_no,drugs_no,masters program,asian,71.0,42318,construction / craftsmanship,straight,religious,m,no,single,NaN
59943,27,athletic,anything,rarely,rarely,noansweredu,asian,73.0,218886,medicine / health,straight,religious,m,rarely,single,NaN


In [5]:
df['height'].unique()


array([75.0, 70.0, 68.0, 71.0, 66.0, 67.0, 65.0, 72.0, 62.0, 64.0, 69.0,
       73.0, 74.0, 60.0, 63.0, 76.0, 61.0, 78.0, 79.0, 59.0, 80.0, 91.0,
       83.0, 77.0, 58.0, 56.0, 95.0, 57.0, 87.0, 81.0, 36.0, 43.0, 52.0,
       55.0, 53.0, 93.0, 8.0, 54.0, 82.0, 3.0, 86.0, 42.0, 84.0, 94.0,
       50.0, 6.0, 47.0, 49.0, 48.0, '65.0', '63.0', '72.0', '74.0',
       '69.0', '71.0', '70.0', '64.0', '67.0', '68.0', '59.0', '60.0',
       '62.0', '73.0', '75.0', '66.0', '76.0', '61.0', '77.0', '57.0',
       '82.0', '95.0', '78.0', '83.0', '79.0', '81.0', '94.0', '84.0',
       '90.0', '58.0', '88.0', 'declined to answer', '80.0', '37.0',
       '9.0', '51.0', '56.0', '91.0', '36.0', '53.0', '1.0', '43.0',
       '92.0', '26.0', '55.0', '48.0', '54.0', '85.0', '89.0', '4.0'],
      dtype=object)

In [6]:
df.drop(df[(df['height']=='declined to answer') ].index, inplace=True)

df

,age,body_type,diet,drinks,drugs,education,ethnicity,height,income,job,orientation,religion,sex,smokes,status,class
0,22,curvy,anything,rarely,drugs_no,noansweredu,asian,75.0,29592,transportation,straight,religious,m,rarely,single,3.0
1,35,average,anything,rarely,drugs_yes,college/university,white,70.0,48630,hospitality / travel,straight,religious,m,no,single,3.0
2,38,skinny,anything,rarely,noanswerdrugs,masters program,noanswerethnicity,68.0,60812,noanswerjob,straight,religious,m,no,available,5.0
3,23,skinny,vegetarian,rarely,noanswerdrugs,noansweredu,white,71.0,18578,student,straight,religious,m,no,single,3.0
4,29,athletic,noanswerdiet,rarely,drugs_no,college/university,asian,66.0,94691,artistic / musical / writer,straight,religious,m,no,single,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59940,59,noanswerbodytype,noanswerdiet,rarely,drugs_no,college/university,noanswerethnicity,62.0,16612,sales / marketing / biz dev,straight,religious,f,no,single,0.0
59941,24,fit,anything,rarely,drugs_yes,noansweredu,white,72.0,118254,entertainment / media,straight,agnostic,m,no,single,NaN
59942,42,average,anything,drinks_no,drugs_no,masters program,asian,71.0,42318,construction / craftsmanship,straight,religious,m,no,single,NaN
59943,27,athletic,anything,rarely,rarely,noansweredu,asian,73.0,218886,medicine / health,straight,religious,m,rarely,single,NaN


In [7]:
df.drop(df[(df['income']=='declined to answer') ].index, inplace=True)

df

,age,body_type,diet,drinks,drugs,education,ethnicity,height,income,job,orientation,religion,sex,smokes,status,class
0,22,curvy,anything,rarely,drugs_no,noansweredu,asian,75.0,29592,transportation,straight,religious,m,rarely,single,3.0
1,35,average,anything,rarely,drugs_yes,college/university,white,70.0,48630,hospitality / travel,straight,religious,m,no,single,3.0
2,38,skinny,anything,rarely,noanswerdrugs,masters program,noanswerethnicity,68.0,60812,noanswerjob,straight,religious,m,no,available,5.0
3,23,skinny,vegetarian,rarely,noanswerdrugs,noansweredu,white,71.0,18578,student,straight,religious,m,no,single,3.0
4,29,athletic,noanswerdiet,rarely,drugs_no,college/university,asian,66.0,94691,artistic / musical / writer,straight,religious,m,no,single,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59940,59,noanswerbodytype,noanswerdiet,rarely,drugs_no,college/university,noanswerethnicity,62.0,16612,sales / marketing / biz dev,straight,religious,f,no,single,0.0
59941,24,fit,anything,rarely,drugs_yes,noansweredu,white,72.0,118254,entertainment / media,straight,agnostic,m,no,single,NaN
59942,42,average,anything,drinks_no,drugs_no,masters program,asian,71.0,42318,construction / craftsmanship,straight,religious,m,no,single,NaN
59943,27,athletic,anything,rarely,rarely,noansweredu,asian,73.0,218886,medicine / health,straight,religious,m,rarely,single,NaN


In [8]:
df.drop(df[(df['age']=='declined to answer') ].index, inplace=True)

df

,age,body_type,diet,drinks,drugs,education,ethnicity,height,income,job,orientation,religion,sex,smokes,status,class
0,22,curvy,anything,rarely,drugs_no,noansweredu,asian,75.0,29592,transportation,straight,religious,m,rarely,single,3.0
1,35,average,anything,rarely,drugs_yes,college/university,white,70.0,48630,hospitality / travel,straight,religious,m,no,single,3.0
2,38,skinny,anything,rarely,noanswerdrugs,masters program,noanswerethnicity,68.0,60812,noanswerjob,straight,religious,m,no,available,5.0
3,23,skinny,vegetarian,rarely,noanswerdrugs,noansweredu,white,71.0,18578,student,straight,religious,m,no,single,3.0
4,29,athletic,noanswerdiet,rarely,drugs_no,college/university,asian,66.0,94691,artistic / musical / writer,straight,religious,m,no,single,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59940,59,noanswerbodytype,noanswerdiet,rarely,drugs_no,college/university,noanswerethnicity,62.0,16612,sales / marketing / biz dev,straight,religious,f,no,single,0.0
59941,24,fit,anything,rarely,drugs_yes,noansweredu,white,72.0,118254,entertainment / media,straight,agnostic,m,no,single,NaN
59942,42,average,anything,drinks_no,drugs_no,masters program,asian,71.0,42318,construction / craftsmanship,straight,religious,m,no,single,NaN
59943,27,athletic,anything,rarely,rarely,noansweredu,asian,73.0,218886,medicine / health,straight,religious,m,rarely,single,NaN


In [29]:
df_test=df.iloc[0:5]
df_test

,age,body_type,diet,drinks,drugs,education,ethnicity,height,income,job,orientation,religion,sex,smokes,status,class
0,22,curvy,anything,rarely,drugs_no,noansweredu,asian,75.0,29592,transportation,straight,religious,m,rarely,single,3.0
1,35,average,anything,rarely,drugs_yes,college/university,white,70.0,48630,hospitality / travel,straight,religious,m,no,single,3.0
2,38,skinny,anything,rarely,noanswerdrugs,masters program,noanswerethnicity,68.0,60812,noanswerjob,straight,religious,m,no,available,5.0
3,23,skinny,vegetarian,rarely,noanswerdrugs,noansweredu,white,71.0,18578,student,straight,religious,m,no,single,3.0
4,29,athletic,noanswerdiet,rarely,drugs_no,college/university,asian,66.0,94691,artistic / musical / writer,straight,religious,m,no,single,6.0


In [31]:
df_test.to_dict(orient='records')

[{'age': 22,
  'body_type': 'curvy',
  'diet': 'anything',
  'drinks': 'rarely',
  'drugs': 'drugs_no',
  'education': 'noansweredu',
  'ethnicity': 'asian',
  'height': 75.0,
  'income': 29592,
  'job': 'transportation',
  'orientation': 'straight',
  'religion': 'religious',
  'sex': 'm',
  'smokes': 'rarely',
  'status': 'single',
  'class': 3.0},
 {'age': 35,
  'body_type': 'average',
  'diet': 'anything',
  'drinks': 'rarely',
  'drugs': 'drugs_yes',
  'education': 'college/university',
  'ethnicity': 'white',
  'height': 70.0,
  'income': 48630,
  'job': 'hospitality / travel',
  'orientation': 'straight',
  'religion': 'religious',
  'sex': 'm',
  'smokes': 'no',
  'status': 'single',
  'class': 3.0},
 {'age': 38,
  'body_type': 'skinny',
  'diet': 'anything',
  'drinks': 'rarely',
  'drugs': 'noanswerdrugs',
  'education': 'masters program',
  'ethnicity': 'noanswerethnicity',
  'height': 68.0,
  'income': 60812,
  'job': 'noanswerjob',
  'orientation': 'straight',
  'religion'

In [9]:
#Categorical variables to use
cat_vars = ["body_type", "diet", "drinks","drugs", "education","ethnicity","job", "orientation","religion","sex", "smokes", "status"]

# Numerical Variables to use
num_vars = ['age', 'income','height']


profile_data = df[cat_vars + num_vars]

In [10]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy="median")
# Num_vars is the list of numerical variables 
profile_num = df[num_vars]
profile_num = imputer.fit_transform(profile_num)

In [11]:
from sklearn.preprocessing import StandardScaler
StandardScaler().fit_transform(profile_num)

array([[-1.09387683, -1.00760278,  1.67839741],
       [ 0.28138861, -0.70754809,  0.42671474],
       [ 0.59875755, -0.51554966, -0.07395832],
       ...,
       [ 1.02191615, -0.80703046,  0.67705128],
       [-0.56492859,  1.97582786,  1.17772434],
       [ 0.7045472 ,  0.11475747, -0.07395832]])

In [12]:
from sklearn.preprocessing import OneHotEncoder
cat_encoder = OneHotEncoder()
profile_cat=df[cat_vars]
profile_cat_hot_encoded = cat_encoder.fit_transform(profile_cat)
profile_cat_hot_encoded


<59942x77 sparse matrix of type '<class 'numpy.float64'>'
	with 719304 stored elements in Compressed Sparse Row format>

In [13]:
# Steps in the ML chain, in order
# Transform gender column to numbers for m to 1 and f to 0

#def changeStatus(sex):
    #if sex == "m":
        #return 1
    #else:
        #return 0
# Along with replace() and map(), this is another way to encode the gender column into numbers.
#df["sex"] = df["sex"].apply(changeStatus)

In [14]:
from sklearn.compose import ColumnTransformer

data_pipeline = ColumnTransformer([
    ('numerical',StandardScaler() , num_vars),
    ('categorical', OneHotEncoder(), cat_vars),
    
])

profile_processed = data_pipeline.fit_transform(df)


In [15]:
# seperate the independent and target variables
X = df.drop(columns=['class'])
X.drop(df.index[59938:59943],0,inplace=True)
y = df['class'].dropna()

print(X,y)

       age         body_type          diet          drinks          drugs  \
0       22             curvy      anything          rarely       drugs_no   
1       35           average      anything          rarely      drugs_yes   
2       38            skinny      anything          rarely  noanswerdrugs   
3       23            skinny    vegetarian          rarely  noanswerdrugs   
4       29          athletic  noanswerdiet          rarely       drugs_no   
...    ...               ...           ...             ...            ...   
59936   32            skinny      anything          rarely  noanswerdrugs   
59937   48           average  noanswerdiet  noanswerdrinks  noanswerdrugs   
59938   52  noanswerbodytype  noanswerdiet          rarely       drugs_no   
59939   31  noanswerbodytype  noanswerdiet          rarely       drugs_no   
59940   59  noanswerbodytype  noanswerdiet          rarely       drugs_no   

                education          ethnicity height  income  \
0           

/var/folders/ng/g2rw737s09b_hmsyrmqzpq300000gn/T/ipykernel_28180/279838899.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X.drop(df.index[59938:59943],0,inplace=True)


In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
model_pipeline = Pipeline(steps=[
                                 ('pre_processing',data_pipeline),
                                 ('random_forest', RandomForestClassifier(random_state=1, n_estimators=500))
                                 ])
# The pipeline can be used as any other estimator
# and avoids leaking the test set into the train set
pipe=model_pipeline.fit(X_train, y_train)
Pipeline(steps=[('pre_processing',data_pipeline),
                ('random_forest', RandomForestClassifier(random_state=1, n_estimators=500))
                ])
model_pipeline.score(X_test, y_test)

0.6427761094427761

In [19]:
X_test

,age,body_type,diet,drinks,drugs,education,ethnicity,height,income,job,orientation,religion,sex,smokes,status
49953,27,athletic,anything,rarely,drugs_yes,college/university,white,74.0,120134,sales / marketing / biz dev,straight,religious,m,no,single
57010,27,skinny,noanswerdiet,rarely,drugs_no,noansweredu,asian,71.0,20451,student,bisexual,religious,m,yes,single
46648,60,fit,anything,rarely,noanswerdrugs,college/university,white,63.0,103786,noanswerjob,straight,religious,f,no,single
28802,43,average,noanswerdiet,rarely,drugs_yes,college/university,white,59.0,162834,sales / marketing / biz dev,straight,religious,f,rarely,single
17874,24,average,vegetarian,rarely,drugs_no,college/university,white,67.0,65986,noanswerjob,gay,atheist,m,no,available
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36871,66,average,anything,rarely,drugs_no,college/university,white,65.0,67831,artistic / musical / writer,straight,religious,f,no,single
44492,19,average,noanswerdiet,drinks_no,drugs_no,noansweredu,black,70.0,384227,medicine / health,bisexual,agnostic,m,no,single
34942,68,noanswerbodytype,noanswerdiet,drinks_no,drugs_no,noansweredu,white,73.0,35776,construction / craftsmanship,straight,religious,m,no,single
59130,21,fit,anything,rarely,drugs_no,noansweredu,white,70.0,25560,student,straight,religious,m,no,seeing someone


In [25]:
y_predict=pd.DataFrame({'age':[27],'body_type': 'athletic', 'diet':'anything', 'drinks':'rarely',
        'drugs':'drugs_yes','education':'college/university','ethnicity':'white','height':[74.0], 'income': [120134], 'job':'sales / marketing / biz dev', 
        'orientation':'straight', 'religion':'religious','sex':'m','smokes':'no','status':'single'})

In [26]:
model_pipeline.predict(y_predict)

array([0.])

In [17]:
model_pipeline.predict(X_test)

array([0., 3., 2., ..., 5., 3., 0.])

In [1]:
model_pipeline

NameError: name 'model_pipeline' is not defined

In [71]:
# Saving model to disk
pickle.dump(model_pipeline, open('model.pkl','wb'))


In [72]:
# Loading model to compare the results
model = pickle.load(open('model.pkl','rb'))
print(model)

Pipeline(steps=[('pre_processing',
                 ColumnTransformer(transformers=[('numerical', StandardScaler(),
                                                  ['age', 'income', 'height']),
                                                 ('categorical',
                                                  OneHotEncoder(),
                                                  ['body_type', 'diet',
                                                   'drinks', 'drugs',
                                                   'education', 'ethnicity',
                                                   'job', 'orientation',
                                                   'religion', 'sex', 'smokes',
                                                   'status'])])),
                ('random_forest',
                 RandomForestClassifier(n_estimators=500, random_state=1))])
